In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


# --- 2) imports ---
import os, glob
import pandas as pd

# --- 3) set paths ---
train_dir  = '/content/drive/My Drive/AI_5/Training_Set/train'
labels_dir = '/content/drive/My Drive/AI_5/Training_Set/labels'
out_dir    = '/content/drive/My Drive/AI_5/Training_Set/labels_yolo'

# make sure output folder exists
os.makedirs(out_dir, exist_ok=True)

# pick up the single CSV in labels folder
csv_files = glob.glob(os.path.join(labels_dir, '*.csv'))
assert len(csv_files)==1, f"Found {len(csv_files)} CSVs in {labels_dir}"
csv_path = csv_files[0]

# --- 4) load and preprocess ---
df = pd.read_csv(csv_path)
# assume columns: filename, width, height, class, xmin, ymin, xmax, ymax

# build class‐to‐ID mapping
classes = sorted(df['class'].unique())
class_map = {c:i for i,c in enumerate(classes)}

# --- 5) convert per‐image ---
for fname, group in df.groupby('filename'):
    base = os.path.splitext(fname)[0]
    out_txt = os.path.join(out_dir, base + '.txt')
    # clear old file if it exists
    if os.path.exists(out_txt):
        os.remove(out_txt)
    with open(out_txt, 'w') as f:
        for _, row in group.iterrows():
            x_min, y_min, x_max, y_max = (row['xmin'], row['ymin'],
                                          row['xmax'], row['ymax'])
            w_img, h_img = row['width'], row['height']
            # compute normalized YOLO bbox
            x_c = ((x_min + x_max)/2) / w_img
            y_c = ((y_min + y_max)/2) / h_img
            bw  = (x_max - x_min)   / w_img
            bh  = (y_max - y_min)   / h_img
            cls_id = class_map[row['class']]
            f.write(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}\n")

print(" All annotations converted to YOLO format in", out_dir)


 All annotations converted to YOLO format in /content/drive/My Drive/AI_5/Training_Set/labels_yolo
